In [2]:
#hide
#default_exp snip
#default_cls_lvl 3
from nbdev.showdoc import show_doc

In [4]:
#export
from nbdev.imports import *

# Snippets Manager

> How to manage snippets (json version)

In [5]:
show_doc(Config, title_level=3)

<h3 id="Config" class="doc_header"><code>class</code> <code>Config</code><a href="https://github.com/fastai/fastcore/tree/master/fastcore/foundation.py#L250" class="source_link" style="float:right">[source]</a></h3>

> <code>Config</code>(**`cfg_path`**, **`cfg_name`**, **`create`**=*`None`*)

Reading and writing `ConfigParser` ini files

In [ ]:
# exporti
from glob import glob
import os
import time
import datetime
import pandas as pd
from io import StringIO
from IPython.core.magic import Magics, magics_class, line_magic, cell_magic
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import HtmlFormatter
from IPython.core.display import HTML, display

from sniptool.core import getmtime_ms, get_change_file

def _snippet_parser(path):
    sys.stdout.write(f"updating snippets from {path}\n")
    sys.stdout.flush()
    notebook = json.load(open(path))

    for cell in notebook["cells"]:
        if not cell["source"]:
            continue
        head = cell["source"][0]
        if head.startswith("# @"):
            desc, *tags = tuple(head.lstrip("# @").split("#"))
            sys.stdout.write(f"snippets: {desc}\n")
            sys.stdout.flush()
            tags = ";".join(tags) if tags else ""
            source = "".join(cell["source"][1:])
            url = ""
            yield (
                dict(
                    tags=tags,
                    desc=desc,
                    source=source,
                    url=url,
                )
            )
# def _get_snippets_file_list(dirs):
#     include = [d for d in dirs if not d.startswith("!")]
#     exclude = [d[1:] for d in dirs if d.startswith("!")]
#     nb_list, links = _get_file_list(include, exclude)
#     mtime = list(map(os.path.getmtime, nb_list))
#     return pd.DataFrame(zip(nb_list, mtime, links), columns=["path", "mtime", "url"])

def _get_file_list(targets, exclude=["__pycache__", ".ipynb_checkpoints"]):
    """ "
    This function recursively prints all contents of a pathlib.Path object
    """
    import configparser

    nb_list = [(None, None)]
    for target in targets:
        target = Path(target)
        cfg1, cfg2 = target / "ghconfig", target / ".git/config"
        if cfg1.is_file():  # has ghconfig
            cfg = yaml.load(open(str(cfg1)), Loader=yaml.BaseLoader)
            root = cfg["repo"].rstrip(".git") + "/" + target.name
        elif cfg2.is_file():
            config = configparser.ConfigParser()
            config.read(str(cfg2))
            root = config['remote "origin"']["url"].rstrip(".git") + "/" + target.name
        else:
            root = None
        for e in exclude:
            if target.name.find(e) != -1:
                return list(map(list, zip(*nb_list)))
        for file in target.iterdir():
            if file.is_dir():
                if "." in file.name:
                    continue
                nb_list += _get_file_list([file.absolute().as_posix()], exclude=exclude)
            elif file.name.endswith(".ipynb"):
                nb_list.append((file.absolute().as_posix(), root))
    # remove invalid lines
    nb_list = list(filter(lambda x: x[1], nb_list))
    return list(map(list, zip(*nb_list)))

def _disp(snippets, short=False):
    display(
        HTML(
            """
    <style>
    {pygments_css}
    </style>
    """.format(
                pygments_css=HtmlFormatter().get_style_defs(".highlight")
            )
        )
    )

    for _, row in snippets.iterrows():
        if short:
            content = row["desc"]
        else:
            content = row["desc"] + "\n" + row["source"][:80]
        display(
            HTML(data=highlight(f"({_})" + content, PythonLexer(), HtmlFormatter()))
        )
        
# test

In [47]:
# exporti
from pathlib import Path
import yaml
import datetime
from tempfile import NamedTemporaryFile

def _get_repo_url(target):
    "# TODO: link to github url"
    import configparser
    cfg1, cfg2 = target / "ghconfig", target / ".git/config"
    if cfg1.is_file():  # has ghconfig
        cfg = yaml.load(open(str(cfg1)), Loader=yaml.BaseLoader)
        repo = cfg["repo"].rstrip(".git") + "/" + target.name
    elif cfg2.is_file():
        config = configparser.ConfigParser()
        config.read(str(cfg2))
        repo = config['remote "origin"']["url"].rstrip(".git") + "/" + target.name
    else:
        repo = None
    return repo
    
def _create_ipynb_list(dirname, root, ext='.ipynb'):
    repo = _get_repo_url(root)
    for target in dirname.iterdir():
        if target.name.endswith(ext):
            mtime = os.path.getmtime(target.name)
            stamp = datetime.datetime.fromtimestamp(mtime, tz=datetime.timezone.utc)
            stamp = int(round(stamp.timestamp()))
#             yield repo + 'blob/master/' + str(target), stamp
            yield str(target), stamp
                
def _create_dir_list(search_dirs, exclude=["__pycache__", ".ipynb_checkpoints"]):
    for target in search_dirs:
        yield Path(target), Path(target)
        for d in Path(target).iterdir():
            if d.is_dir():
                for e in exclude:
                    if d.name.find(e) != -1:
                        continue
                if "." in d.name: continue
                yield d, Path(target)

def _snapshot_taken():
    f = NamedTemporaryFile(mode="w", suffix=".txt", delete=False)
    data = []
    for dirname, root in _create_dir_list('.'):
        for url, stamp in _create_ipynb_list(dirname, root):
            data.append((url, stamp))
    (pd.DataFrame(data, columns =['url', 'stamp'])
        .sort_values(by='url')
        .to_csv(f, sep='|', index=False, header=False))
    return f.name
    
def _detech_changed_files(last='last_snip.txt'):
    if not os.path.isfile(last):
        os.system(f'touch {last}')
    this = _snapshot_taken()
#     print(f'diff -y --suppress-common-lines {last} {this}')
    diff_ret = os.popen(f'diff -y --suppress-common-lines {last} {this}').read()
    for line in diff_ret.splitlines():
        file = line.split('|')[0].split()[-1]
        yield file
    os.rename(this, last)

02_snip.ipynb


In [44]:
for path in _detech_changed_files():
    print(path)

https://github.com/suredream/sniptool/blob/master/00_core.ipynb|1649603769
https://github.com/suredream/sniptool/blob/master/01_viz.ipynb|1649579529
https://github.com/suredream/sniptool/blob/master/02_snip.ipynb|1649609400
https://github.com/suredream/sniptool/blob/master/03_ver.ipynb|1649578070
https://github.com/suredream/sniptool/blob/master/04_misc.ipynb|1649604042
https://github.com/suredream/sniptool/blob/master/docs/index.ipynb|1649566371
https://github.com/suredream/sniptool/blob/master/index.ipynb|1649566371


In [4]:
# exporti
@magics_class
class SnippetsMagics(Magics):
    """snip magic for snippets management.
    Provides the %snip magic.
    """
    def __init__(self, shell):
        super(SnippetsMagics, self).__init__(shell=shell)
        app_dir = os.path.dirname(os.path.abspath(__file__))
        app_dir = os.path.dirname(app_dir)
        self.jsonfile = f'{app_dir}/snip.json'
        self.snapshot = f'{app_dir}/last_snip.txt'
        try:
            self.df = pd.read_json(self.jsonfile)
        except ValueError:
            self.df = pd.DataFrame()

    @line_magic
    def sn(self, parameter_s=""):
        """snippets management

        Usage:
        * ``%snip``          - total numbers of all snippets and notebooks
        * ``%snip bar``      - search in the db
        * ``%snip -r``       - reset indexing
        * ``%snip -u``       - update db
        * ``%snip <key >-s``       - short output

        New search directories can be inserted into config.yaml['search_dirs']
        """
        opts, argsl = self.parse_options(parameter_s, "drust", mode="string")
        args = argsl.split()
        if "u" in opts: # update
            if "r" in opts:
                if os.path.isfile(self.snapshot):
                    os.remove(self.snapshot)
            nestedList = map(_snippet_parser, list(_detech_changed_files(self.snapshot)))
            ret_df = pd.DataFrame([item for sublist in nestedList for item in sublist])
            if self.df.empty:
                self.df = ret_df
            else:
#                 self.df = self.df.merge(ret_df, on='desc', how='right')
                self.df = pd.concat([self.df, ret_df], axis=0, join='outer')
            self.df.to_json(self.jsonfile, orient='records')
        elif "d" in opts:  # display first
            self.df.head().pipe(_disp, short="s" in opts)
        elif not args: # info
            try:
                n_ipynb = os.popen(f'wc -l {self.snapshot}').read().split()[0]
            except:
                n_ipynb = 0
            print(
                "{} snippets, {} notebooks in SnippetDB".format(
                    self.df.shape[0], n_ipynb
                )
            )
        elif args[-1].isdigit():  # load snippets
            pos, q = int(args[-1]), " ".join(args[:-1])
            cell = (
                self.df.query(f"desc.str.contains('{q}')", engine="python")
                .pipe(lambda _df: _df.iloc[pos]["source"]))
            self.shell.set_next_input(cell, replace=False)
        else:
            q = " ".join(args)
            (
                self.df.query(f"desc.str.contains('{q}')", engine="python")
                .reset_index()
                .pipe(_disp, short="s" in opts)
            )

In [61]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [65]:
%reload_ext sniptool